In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.1
)

## from_template 메소드
* 변수를 중괄호로 묶어서 템플릿에 정의

In [4]:
from langchain_core.prompts import PromptTemplate

# template 정의, {}안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요?"

# from_ template 메소드를 사용해 PromptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [5]:
# prompt 생성(완성) format() 메소드를 이용해 변수에 값을 넣음
prompt = prompt.format(name ="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [12]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate.from_template(template)

print(prompt)

chain = prompt | llm

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


In [10]:
# 기본적으로 딕셔너리 형태로 변수를 작성해야하지만
# 변수가 한개일 때는 변수만으로 작성이 가능

# print(chain.invoke({"language": "Python"}).content)
print(chain.invoke("Python").content) # 위와 동일

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1989년에 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성을 중요시하며, 프로그래밍을 보다 쉽게 할 수 있도록 설계했습니다. Python은 1991년에 첫 번째 버전이 공개되었고, 이후로 많은 발전을 거듭하며 현재까지 널리 사용되고 있는 프로그래밍 언어가 되었습니다.


### PromptTemplate 객체 생성과 동시에 prompt 생성
* input_variables 인자를 사용해 변수를 지정한다.
* 템플릿에 작성한 변수가 input_variableds에 없으면 예외를 발생시켜준다.

In [14]:
# template 정의
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"],
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

**partial_ variables**
* 연산중 미리 계산된 변수를 프롬프트 템플릿에 지정해 넣을 수 있다.
* 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 사용
ex) 날짜, 시간

In [15]:
# template 정의
template ="{language1}과 {language2}는 각각 누가 만들었나요?"

# PrompteTemplate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2" : "Java" # dictionary 형태로 partial_variables 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Java'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [16]:
prompt.format(language1 = "Python")

'Python과 Java는 각각 누가 만들었나요?'

### partial()
* 연산중에 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을 수 있다.

In [17]:
prompt_partial = prompt.partial(language2 = "JavaScript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [18]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과 JavaScript는 각각 누가 만들었나요?')

In [19]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 Python을 만들 때 코드의 가독성을 중요시하며, 프로그래밍 언어의 사용을 쉽게 하려는 목표를 가지고 있었습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 넷스케이프(Netscape)에서 일할 당시 웹 브라우저에서 동적인 기능을 추가하기 위해 JavaScript를 만들었습니다. 처음에는 "Mocha"라는 이름으로 시작했지만, 이후 "LiveScript"를 거쳐 "JavaScript"로 이름이 변경되었습니다.


### 파일로부터 template 읽어오기
* prompt를 편하게 작성 및 수정
* 상황에 맞게 파일을 작성해두면 필요할때마다 꺼내서 쓸 수 있다.

In [24]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/language_simple.yaml", encoding="utf-8")

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에서 3줄 설명하기'


In [25]:
prompt.format(language="Python")

'Python 언어에서 3줄 설명하기'

In [26]:
prompt2 = load_prompt("prompts/language.yaml", encoding="utf-8")
print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요.
언어의 특징을 다음의 양식에 맞게 정리하세요.
300자 내외로 작성하세요.
한글로 작성하세요.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [29]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Python")
print(answer)

1. 특징: Python은 간결하고 가독성이 높은 문법을 가진 고급 프로그래밍 언어로, 다양한 프로그래밍 패러다임(객체지향, 함수형 등)을 지원합니다. 동적 타이핑과 자동 메모리 관리를 제공하여 개발 속도를 높이고, 풍부한 라이브러리와 생태계를 갖추고 있습니다.

2. 제작자: Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다.

3. 대표적인 프레임워크: Django, Flask, FastAPI 등이 있으며, 웹 개발에 많이 사용됩니다.

4. 많이 사용되는 분야: 데이터 과학, 인공지능, 웹 개발, 자동화 스크립트, 게임 개발 등 다양한 분야에서 널리 사용됩니다.


### ChatPromptTemplate
* 대화 목록을 프롬프트로 주입하고자 할 때 사용할 수 있다.
* 메세지는 튜플형태로 전달
    * ("role","message") 구성되고, 리스트 형태로 생성 가능

**role**
* sysmtem : 시스템 설정메세지로 주로 전역설정을 할 때 사용
* human : 사용자의 입력 메세지
* ai : AI의 답변 메시지

In [30]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [31]:
print(chat_prompt.format(language="python"))

Human: python의 제작자는 누구인가요?


In [35]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai","안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)

messages = chat_template.format_messages(
    name="구찌", user_input= "당신의 이름은 무엇입니까?"
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트 입니다. 당신의 이름은 구찌입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]


In [36]:
print(llm.invoke(messages).content)

제 이름은 구찌입니다! 당신과 대화하게 되어 기쁩니다. 어떤 이야기를 나눌까요?


### MessagesPlaceHolder
* 아직 확정된 메세지가 아니지만, 나중에 채워질 메세지 위치를 잡아두기 위해 사용한다.
* 보통 대화 기록을 하고싶을 때 사용한다.

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴스입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다."
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count}단어로 요약합니다.")
        
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [41]:
formatted_chat_prompt =chat_prompt.format(
    word_count = 5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴스입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5단어로 요약합니다.


In [42]:
chain = chat_prompt | llm | StrOutputParser()

In [44]:
input = {
    "word_count" : 5,
    "conversation":[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

chain.invoke(input)

'파이썬, 리스트, 중복 제거, set.'

In [45]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 33.7 MB/s eta 0:00:00


In [46]:
from langchain_community.llms import Ollama

#model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

response

C:\Users\20111\AppData\Local\Temp\ipykernel_13804\2235018838.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2:9b")


' 지구의 자전 주기는 약 24시간입니다. 이것은 태양 아래에서 하루가 끝나는 시간으로 알려져 있습니다.\n\n\n그러나, 지구가 중력과 질량을 가진 다른 천체와 상호 작용하기 때문에 실제 자전 주기는 정확히 24시간이 아닙니다. \n\n* **시공간의 변화:** 우주에서 관측할 때, 지구의 자전은 약간씩 변합니다. 태양과 달의 중력에 의해 지구의 자전 속도가 천천히 변하고 있습니다.\n* **축 기울기:** 지구의 축이 평면과 이루는 각도인 축 기울기도 변하며, 이로 인해 극과 적도 사이의 온도 차이가 발생합니다.\n\n따라서 지구 자전 주기를 정확히 계산하기 위해서는 위와 같은 요소들을 고려해야 합니다.\n\n\n그러나 일반적으로 우리는 하루를 24시간으로 알고 있으며, 이것이 지구 자전 주기의 가장 일반적인 기준입니다.'

In [48]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model ="gemma2:9b",
    temperature=0.1
)


C:\Users\20111\AppData\Local\Temp\ipykernel_13804\1337631746.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [49]:
chain = chat_prompt | model | StrOutputParser()

chain.invoke(input)

'파이썬, 중복 제거, set 사용, 간단 해결, 유용  \n'